In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from tqdm import tqdm 

In [2]:
# 0.장치(device) 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')



Using device: cuda


In [3]:
# 1. 데이터 준비
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

training_set = torchvision.datasets.FashionMNIST( #Dataset 생성
    './data',
    train=True,
    download=True,
    transform = transform
)
training_dataloader = DataLoader(
    training_set,batch_size=64, shuffle=True,
)


In [11]:
#2.1 모델 생성
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28*28,512), # 한장씩 내적x, 한방에 다 내적됨
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,10)
        )
    def forward(self,x):
        x = self.flatten(x) 
        # (64,1,28,28) -> (64,784) : 4차원이 2차원으로 준다.
        # 원래 차원이   
        return self.layers(x)



In [12]:
# 다만든 신경망을 GPU로 보내야함
model = MLP().to(device) # model을 cuda로 보내겠다.

In [13]:
# 3. 다중 분류 
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [ ]:
# 훈련 loop 돌리기
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0

    # 지금부터 중요!! 모델을 훈련 모드로 설정.Dropout, BatchNorm
    model.train()  # <주의>훈련모드를 설정하는거임, 훈련하는건 아님
                   # <선언>지금부터 훈련할거야~~~를 알리는 거임
    progress_bar = tqdm(training_dataloader,desc=f'Epoch {epoch+1}/{num_epochs}')

    for i,data in enumerate(progress_bar): # <중요>진행되는 바 보여줌
        inputs, labels = data
        #<중요> 데이터를 메모리에 넣어뒀다가 학습에 필요할때만 cuda에 넣겠다!!!
        inputs, labels = inputs.to(device),labels.to(device)


Epoch 1/5:  53%|█████▎    | 494/938 [00:05<00:05, 88.54it/s]


KeyboardInterrupt: 